In [1]:
from bs4 import BeautifulSoup
import urllib.request
import re

url="http://www.leagueofgraphs.com/champions/stats/diamond"
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page.read(), "lxml")



In [2]:
for tr in soup.find_all('tr')[1:145]:
    tds = tr.find_all('td')
    
    if len(tds) >1:
        name = tds[1].text.split()
    
        champ_name = name[0].lower().replace("'", '').replace(",", '').replace(".", '')
        if champ_name == 'lee':
            champ_name = 'leesin'
        elif champ_name == 'miss':
            champ_name = 'missfortune'
        elif champ_name == 'twisted':
            champ_name = 'twistedfate'
        elif champ_name == 'master':
            champ_name = 'masteryi'
        elif champ_name == 'xin':
            champ_name = 'xinzhao'
        elif champ_name == 'dr':
            champ_name = 'drmundo'
        elif champ_name == 'tahm':
            champ_name = 'tahmkench'
        elif champ_name == 'aurelion':
            champ_name = 'aurelionsol'              
        elif champ_name == 'jarvan':
            champ_name = 'jarvaniv' 
            
        pop = float(tds[2].text.split()[0][0:-1])
        winrate = tds[3].text.split()[0][0:-1]
        banrate = tds[4].text.split()[0][0:-1]
        
#         print(champ_name, str(pop), str(winrate), str(banrate))
        
        with open('lolcurrent_raw.csv', 'a') as f:
            s = champ_name + ',' + str(pop) + ',' +  str(winrate) + ',' + \
                str(banrate) + '\n'
            f.write(s)

In [8]:
import pandas as pd
from datetime import datetime
import time

df = pd.read_csv('lolcurrent_raw.csv', names = ['name', 'popularity', 'winrate',
                                           'banrate'])
df_release = pd.read_csv('lolrelease.csv', names=['name', 'release'])

In [9]:
df = df.merge(df_release, how='outer', on='name')

In [10]:
df.dropna(inplace=True, axis=0)

In [11]:
df.head()

,name,popularity,winrate,banrate,release
0,leesin,34.6,49.1,10.7,1.270080e+09
1,leesin,33.9,49.3,10.0,1.270080e+09
2,ezreal,32.9,48.8,0.2,1.267402e+09
3,ezreal,32.8,48.9,0.2,1.267402e+09
4,jhin,31.3,50.4,0.5,1.451606e+09


In [12]:

date = datetime.now()
date = time.mktime(date.timetuple())

df['tsr'] = df.apply(lambda x: date-x.release, axis=1)

In [13]:
df.head()

,name,popularity,winrate,banrate,release,tsr
0,leesin,34.6,49.1,10.7,1.270080e+09,215457465.0
1,leesin,33.9,49.3,10.0,1.270080e+09,215457465.0
2,ezreal,32.9,48.8,0.2,1.267402e+09,218135865.0
3,ezreal,32.8,48.9,0.2,1.267402e+09,218135865.0
4,jhin,31.3,50.4,0.5,1.451606e+09,33931065.0


In [231]:
df.to_csv('lolcurrent.csv', index=False)

In [ ]:
sudo docker run --rm -d --name flask1 -p 80:80 joe/flaskimage:version002

In [ ]:
sudo docker stop flask1
sudo docker build -t joe/flaskimage:version002 .